#### pip dependencies
- pymongo

#### other dependencies
- https://github.com/veryhappythings/dotamatch (modified version)

## Get latest matches from steam API

In [1]:
import pprint
import json, jsonpickle
from dotamatch import get_key
from dotamatch.history import MatchHistoryBySequenceNum
from dotamatch.players import PlayerSummaries
from dotamatch.heroes import Heroes

key = get_key()
history = MatchHistoryBySequenceNum(key)

In [2]:
#1100000000 come from bruce forcing match_seq_num in api
for match in history.matches(start_at_match_seq_num = 1100000000):
    latest = match

In [63]:
latest.match_seq_num
#data = json.loads(latest)

1100000202

## Connect to MongoDB

In [3]:
from pymongo import MongoClient, ASCENDING
client = MongoClient('localhost', 3001)

In [4]:
db = client.meteor
matches = db.matches

In [21]:
matches.create_index([("match_id", ASCENDING)], unique=True)

u'match_id_1'

In [16]:
latest.herolist = []
for player in latest.players:
    latest.herolist.append(player["hero_id"])

In [17]:
ldict = latest.__dict__

In [18]:
ldict.pop("parent", None)

In [19]:
ldict

{'barracks_status_dire': 0,
 'barracks_status_radiant': 63,
 'cluster': 192,
 'duration': 2772,
 'engine': 0,
 'first_blood_time': 194,
 'flags': 0,
 'game_mode': 1,
 'herolist': [96, 19, 6, 7, 51, 11, 33, 71, 23, 68],
 'human_players': 10,
 'leagueid': 0,
 'lobby_type': 0,
 'match_id': 1226990179,
 'match_seq_num': 1100000202,
 'negative_votes': 0,
 'players': [{u'account_id': 108090943,
   u'assists': 13,
   u'deaths': 8,
   u'denies': 7,
   u'gold_per_min': 393,
   u'hero_id': 96,
   u'item_0': 1,
   u'item_1': 127,
   u'item_2': 90,
   u'item_3': 214,
   u'item_4': 114,
   u'item_5': 0,
   u'kills': 8,
   u'last_hits': 108,
   u'leaver_status': 0,
   u'level': 20,
   u'player_slot': 0,
   u'xp_per_min': 487},
  {u'account_id': 4294967295,
   u'assists': 13,
   u'deaths': 2,
   u'denies': 5,
   u'gold_per_min': 715,
   u'hero_id': 19,
   u'item_0': 48,
   u'item_1': 147,
   u'item_2': 112,
   u'item_3': 116,
   u'item_4': 108,
   u'item_5': 172,
   u'kills': 11,
   u'last_hits': 367

In [20]:
matches.insert_one(ldict)

   ## Create / Update Sequence Data

In [40]:
preferences = db.preferences

In [46]:
# only run this first time
preferences.insert_one({'type': "settings", 'match_sequence_num': 1100000000})

In [51]:
preferences.find_one()["match_sequence_num"]

123

In [52]:
preferences.update_one({"type": "settings"}, {"$set": {"match_sequence_num": 1100000000}})